<a href="https://colab.research.google.com/github/Programapython/detector_osteoartritis/blob/main/practicas/practica1/practica_1_(pytorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalación de pytorch

In [1]:
!pip install torch
!pip install torchvision

##Importación de dependencias

In [2]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

##Definición de hiperparámetros

In [3]:
input_size = 784
hidden_size = 128
num_classes = 10
num_epochs = 5
batch_size = 100
lr = 1e-3

##Descargamos la base de datos

In [8]:
transform = transforms.Compose([transforms.ToTensor()])
train_data = dsets.FashionMNIST('~/.pytorch/FashionMNIST_data/', download=True, train=True, transform=transform)
test_data = dsets.FashionMNIST('~/.pytorch/FashionMNIST_data/', download=True, train=True, transform=transform)

100%|██████████| 26421880/26421880 [00:09<00:00, 2794316.94it/s]


Extracting /root/.pytorch/FashionMNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/FashionMNIST_data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 193099.42it/s]


Extracting /root/.pytorch/FashionMNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/FashionMNIST_data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3589000.07it/s]


Extracting /root/.pytorch/FashionMNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/FashionMNIST_data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 20142049.43it/s]

Extracting /root/.pytorch/FashionMNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/FashionMNIST_data/FashionMNIST/raw



##Leemos la data

In [9]:
train_gen = torch.utils.data.DataLoader(dataset = train_data,
                                             batch_size = batch_size,
                                             shuffle = True)

test_gen = torch.utils.data.DataLoader(dataset = test_data,
                                      batch_size = batch_size,
                                      shuffle = False)

##Definición del modelo

In [10]:
class Net(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.drop = nn.Dropout(0.2)
    self.fc2 = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    out = self.fc1(x)
    out = self.relu(out)
    out = self.drop(out)
    out = self.fc2(out)
    return out

##Instanciación del modelo

In [11]:
net = Net(input_size, hidden_size, num_classes)

if torch.cuda.is_available():
  net.cuda()

##Compilación del modelo

In [12]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr)

#Entrenamiento

In [13]:
for epoch in range(num_epochs):
  for i, (images, labels) in enumerate(train_gen):
    images = images.cuda()
    print(images)
    labels = labels.cuda()
    images = images.view(-1, 28 * 28)
    print(labels)

    optimizer.zero_grad()
    outputs = net(images)
    print(outputs)
    loss = loss_function(outputs, labels)
    loss.backward()
    optimizer.step()

    if (i + 1) % 100 == 0:
        print('Epoca [%d/%d], Step [%d/%d], Loss: %.4f'
              % (epoch + 1, num_epochs, i + 1, len(train_data) // batch_size, loss.item()))

Se han truncado las últimas 5000 líneas del flujo de salida.
         -1.8626e+00, -2.2923e+00, -2.8716e+00,  6.8145e+00, -7.5691e+00],
        [ 9.2009e+00, -1.0218e+01, -2.5924e+00,  1.5382e+00, -9.3885e+00,
         -1.1065e+01,  5.1407e+00, -1.8388e+01, -6.5615e+00, -1.8630e+01],
        [-2.2913e+00, -5.1537e+00,  4.1372e+00, -2.5725e+00,  4.0681e+00,
         -9.0609e+00,  1.6530e+00, -8.1149e+00, -3.3135e+00, -1.2772e+01],
        [ 1.2962e+00, -5.4225e+00, -3.7548e+00,  4.3052e+00, -4.1488e+00,
         -4.1903e+00, -7.0731e-01, -8.5056e+00, -6.8031e-01, -7.9235e+00],
        [-2.2448e-02,  1.1796e+00, -1.2507e+00,  2.5916e-01, -3.0010e+00,
         -4.9421e+00,  5.6557e-01, -2.1227e+00, -1.7708e+00, -3.7818e+00],
        [-1.2115e+00,  1.4904e+01, -1.4218e+00,  5.8570e-01, -1.5718e+00,
         -1.2010e+01, -8.0051e+00, -1.1650e+01, -1.0776e+01, -1.8170e+01],
        [-1.1326e-01, -3.3802e+00,  2.3911e+00,  9.6193e-01,  4.0481e+00,
         -1.2275e+01,  2.4296e+00, -1.2041e+0

In [14]:
correct = 0
total = 0
for images, labels in test_gen:
    images = images.view(-1, 28*28).cuda()
    labels = labels.cuda()

    output = net(images)
    _, predicted = torch.max(output, 1)
    correct += (predicted == labels).sum()
    total += labels.size(0)

print('Accuracy: %.3f %%' % (100 * correct / (total + 1)))

Accuracy: 87.859 %
